In [1]:
import requests
import time
import json
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from html import unescape

headers = {"user-agent" : "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

def download(method, url, param=None, data=None, timeout=1 , maxretires=3):
    try : 
        resp = requests.request(method, url, params=param, data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e :
        if 500 <= e.response.status_code < 600 and maxretires > 0:
            print(maxretires)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretires-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

# 인터파크

In [235]:
driver = webdriver.Chrome()
driver.get("http://ticket.interpark.com")

In [200]:
%%writefile interpark_login.json
{
    "id":"cdetgfr005",
    "pw":"*wnsldj13*"
}

Overwriting interpark_login.json


In [201]:
with open("interpark_login.json") as fp:
    interpark_login = json.load(fp)

In [27]:
# 로그인 전 쿠키 상태 확인.
_logout = [_["name"] for _ in driver.get_cookies()]

In [202]:
# 로그인 버튼 찾기
login = driver.find_element_by_css_selector("#logstatus > a")
if login and login.get_attribute("id") == "aLogin":
    driver.get(login.get_attribute("href"))
    driver.find_element_by_css_selector("#UID").send_keys(interpark_login["id"])
    driver.find_element_by_css_selector("#PWD").send_keys(interpark_login["pw"])
    driver.find_element_by_css_selector("div.loginBtn > button").click()
else:
    print("이미 로그인 상태")

In [196]:
# 위 코드를 함수로 만들기
def login(interpark_login, method="login"):
    try:
        login = driver.find_element_by_css_selector("#logstatus > a")
        if login and login.get_attribute("id") == "aLogin":
            driver.get(login.get_attribute("href"))
            driver.find_element_by_css_selector("#UID").send_keys(interpark_login["id"])
            driver.find_element_by_css_selector("#PWD").send_keys(interpark_login["pw"])
            driver.find_element_by_css_selector("div.loginBtn > button").click()
        else:
            login.click()
    except:
        return False
    return True

In [23]:
login(interpark_login, "login")

True

In [26]:
login(interpark_login, "logout")

True

In [24]:
# 로그인 후 쿠키 상태 확인.
_login = [_["name"] for _ in driver.get_cookies()]

In [28]:
# 쿠키 길이 확인
len(_login), len(_logout)

(40, 22)

In [197]:
# 로그인 상태면 로그아웃, 로그아웃 상태면 로그인.
if "ISession%5FID" not in [_["name"] for _ in driver.get_cookies()]:
    login(interpark_login)
else:
    login(None, "logout")

In [31]:
for _ in driver.find_elements_by_css_selector("ul.gnbBtns a[class^=btn]"):
    print(_.find_element_by_css_selector("img").get_attribute("alt"))

전체메뉴보기
뮤지컬
콘서트
연극
클래식/무용
스포츠
레저
전시/행사
아동/가족
TOPING
랭킹
지역
공연장


In [203]:
# 위 코드 함수 만들기
def gnbbtns(category="콘서트"):
    links = [_ for _ in driver.find_elements_by_css_selector("ul.gnbBtns a[class^=btn]")]
    titles = [_.find_element_by_css_selector("img").get_attribute("alt") for _ in links]
    
    if category in titles:
        return links[titles.index(category)].get_attribute("href")
    return False

In [204]:
url = gnbbtns("콘서트")
if url:
    driver.get(url)

In [40]:
driver.find_elements_by_css_selector("div.wrap_issue dl.list > dt.issue_obj > a")

[<selenium.webdriver.remote.webelement.WebElement (session="d0e7ae03694f344102d37ed04183295f", element="8b095e14-2d3f-4fc0-9f5d-0f0d52b48384")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d0e7ae03694f344102d37ed04183295f", element="bbf0e464-c6f1-4cf2-ba3a-99b7773c0535")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d0e7ae03694f344102d37ed04183295f", element="1c87ddd7-f181-47ee-bed9-de7b490d016f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d0e7ae03694f344102d37ed04183295f", element="18316ce6-e6af-432f-b229-649199d09fee")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d0e7ae03694f344102d37ed04183295f", element="7defe839-d6ca-4126-8792-6317b68e5ff5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d0e7ae03694f344102d37ed04183295f", element="b2221fb2-b43d-42fd-8d39-631c7b64bc94")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d0e7ae03694f344102d37ed04183295f", element="929fe73e-6475-4eb0-b33e-35

In [205]:
# 위 코드 함수로.
def hotissue(issue=None):
    issues = [_ for _ in driver.find_elements_by_css_selector("div.wrap_issue dl.list > dt.issue_obj > a")]
    title = [_.find_element_by_css_selector("span.txt1").text for _ in issues]
    
    if issue in title:
        return issues[title.index(issue)].get_attribute("href")
    else:
        return issues[0].get_attribute("href")

In [206]:
url = hotissue("박정현")

In [207]:
driver.get(url)

In [242]:
# 선생님 버전. 선택할 수 있는 날짜 선택.
def bookWindow(options=None):
    if options:
        #iframe = driver.find_element_by_css_selector("#ifrCalendar")
        iframe = driver.find_element_by_name("ifrCalendar")
        
        if iframe:
            driver.switch_to_frame(iframe)
            dates = [_ for _ in driver.find_elements_by_css_selector("td.able > a")]
            title = [_.text for _ in dates]
    
        if options in title :
            driver.execute_script(dates[title.index(options)].get_attribute("onclick"))
        else :
            driver.execute_script(dates[0].get_attribute("onclick")) #제일 앞 날짜

        driver.switch_to_default_content()
    
    driver.find_element_by_css_selector(".tk_dt_btn_TArea > a.btn_rev").click()
    driver.switch_to_window(driver.window_handles[-1])
    
    #driver.find_element_by_css_selector("#LargeNextBtnImage").click()

In [243]:
driver.switch_to_window(driver.window_handles[0])

C:\Users\cdetg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [217]:
driver.window_handles

['CDwindow-529BD0DE189FD9498500D763BDE2419C']

In [244]:
bookWindow(2)

C:\Users\cdetg\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.frame instead
  
C:\Users\cdetg\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: use driver.switch_to.default_content instead
C:\Users\cdetg\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: use driver.switch_to.window instead


In [155]:
driver.close()

In [178]:
driver.switch_to_default_content()

C:\Users\cdetg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [106]:
# 안전하게 창 닫기 함수.
def safeClose():
    if len(driver.window_handles) > 1:
        for _ in driver.window_handles[1:]:
            driver.switch_to_window(_)
            driver.close()
        driver.switch_to_window(driver.window_handles[0])
        driver.switch_to_default_content()

In [245]:
driver.close()